<a href="https://colab.research.google.com/github/wkotala/DNN-HW1/blob/main/DNN-homework-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project: Multitask Learning for Geometric Shape Classification and Counting**

In [ ]:
test